#Reduced Dataset

## python imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt  # To visualize
import pandas as pd  # To read data
from sklearn import linear_model, metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

seed = 13
kfold = KFold(n_splits=3, shuffle=True, random_state=seed)

def bestParams(algorithm, hp_candidates):
  grid = GridSearchCV(estimator=algorithm, param_grid=hp_candidates, cv=kfold, scoring='r2')
  grid.fit(X_train, y_train)
  return grid

##Mount Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Reading Data

In [4]:
data = pd.read_csv('/content/drive/MyDrive/AV_V/all_and_classification.csv')  # load data set


data = data.drop('run', axis=1)
data['forks'] = data['forks'].replace(['steady state'], 1)
data['forks'] = data['forks'].replace(['no steady state'], 0)
data = data.loc[data['forks'] == 1]
data = data.drop('forks', axis=1)
data = data.loc[data['steady_state_starts']<= 625]
data


,0,1,2,3,4,5,6,7,8,9,...,2992,2993,2994,2995,2996,2997,2998,2999,steady_state_starts,file_name
1,0.188219,0.075891,0.073269,0.073335,0.073138,0.074449,0.104530,0.079888,0.071762,0.072090,...,0.073138,0.073204,0.073859,0.073204,0.073466,0.073269,0.073400,0.073597,480,eclipse__eclipse-collections#org.eclipse.colle...
2,0.190317,0.075170,0.072417,0.072221,0.073073,0.073531,0.113967,0.096076,0.088605,0.087949,...,0.076218,0.074908,0.074908,0.074842,0.073925,0.075497,0.073925,0.074514,421,eclipse__eclipse-collections#org.eclipse.colle...
4,0.185860,0.074908,0.072155,0.072942,0.072810,0.074252,0.104530,0.095683,0.087753,0.085721,...,0.072745,0.076349,0.072745,0.072614,0.074056,0.072417,0.071959,0.073925,385,eclipse__eclipse-collections#org.eclipse.colle...
6,0.191103,0.077136,0.074187,0.073531,0.072614,0.074121,0.117703,0.094700,0.088146,0.085656,...,0.076808,0.076481,0.073794,0.075366,0.073662,0.074908,0.074187,0.073531,408,eclipse__eclipse-collections#org.eclipse.colle...
7,0.188219,0.074646,0.072876,0.073073,0.073925,0.074252,0.090472,0.094634,0.087359,0.086114,...,0.074187,0.074514,0.075104,0.075104,0.074514,0.075563,0.074711,0.074646,433,eclipse__eclipse-collections#org.eclipse.colle...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5848,0.000042,0.000024,0.000031,0.000031,0.000031,0.000031,0.000031,0.000031,0.000031,0.000031,...,0.000017,0.000017,0.000018,0.000018,0.000018,0.000018,0.000018,0.000018,53,apache__logging-log4j2#org.apache.logging.log4...
5849,0.000045,0.000023,0.000031,0.000030,0.000030,0.000030,0.000030,0.000030,0.000030,0.000030,...,0.000017,0.000017,0.000017,0.000018,0.000017,0.000017,0.000017,0.000017,321,apache__logging-log4j2#org.apache.logging.log4...
5851,0.001629,0.002516,0.002812,0.002784,0.002766,0.002760,0.003237,0.001095,0.001196,0.001301,...,0.001093,0.001142,0.001118,0.001779,0.002299,0.001127,0.001089,0.001095,395,yellowstonegames__SquidLib#squidpony.performan...
5855,0.001660,0.002597,0.002875,0.002770,0.002834,0.002764,0.002709,0.001095,0.001221,0.001445,...,0.001089,0.001090,0.001092,0.001093,0.001092,0.001106,0.001109,0.001106,190,yellowstonegames__SquidLib#squidpony.performan...


##Dividing data (train and test) based on file name 

In [5]:
data_file_name = data['file_name']

In [6]:
data_file_name_unique = data_file_name.drop_duplicates(keep="first")

In [7]:
from sklearn.model_selection import train_test_split
data_file_name_train, data_file_name_test = train_test_split(data_file_name_unique, test_size=0.3,
                                                    random_state=1)

In [8]:
train_data = pd.merge(data, data_file_name_train, left_on='file_name', right_on='file_name')

In [9]:
test_data = pd.merge(data, data_file_name_test, left_on='file_name', right_on='file_name')

In [10]:
train_data = train_data.drop('file_name', axis=1)
test_data = test_data.drop('file_name', axis=1)

In [11]:
X_train = train_data.drop('steady_state_starts', axis=1) 
y_train  = train_data['steady_state_starts']
X_test = test_data.drop('steady_state_starts', axis=1)
y_test = test_data['steady_state_starts']

### Standardizing the data

In [12]:
sc = StandardScaler()
sc.fit(X_train.T)
X_train = pd.DataFrame(sc.transform(X_train.T))
X_train = X_train.T

In [13]:
sc = StandardScaler()
sc.fit(X_test.T)
X_test = pd.DataFrame(sc.transform(X_test.T))
X_test = X_test.T

##Finding best hyperparameter values

In [14]:
from sklearn.linear_model import LogisticRegression

In [17]:
from sklearn.linear_model import LogisticRegression
softmax_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs')
softmax_reg.fit(X_train, y_train)
y_pred_sofmax = softmax_reg.predict(X_test)

softmax_mae = mean_absolute_error(y_test, y_pred_sofmax)
softmax_mse = mean_squared_error(y_test, y_pred_sofmax)
softmax_rmse = mean_squared_error(y_test, y_pred_sofmax, squared=False)
softmax_r2 = r2_score(y_test, y_pred_sofmax)

print("MAE:", softmax_mae)
print("MSE:", softmax_mse)
print("RMSE:", softmax_rmse)
print("R2 Score:", softmax_r2)

MAE: 102.47030497592296
MSE: 30549.879614767255
RMSE: 174.78523854939024
R2 Score: -0.38262178505808087


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [19]:
from sklearn.linear_model import LogisticRegression
softmax_reg = LogisticRegression(multi_class='multinomial', solver='sag')
softmax_reg.fit(X_train, y_train)
y_pred_sofmax = softmax_reg.predict(X_test)

softmax_mae = mean_absolute_error(y_test, y_pred_sofmax)
softmax_mse = mean_squared_error(y_test, y_pred_sofmax)
softmax_rmse = mean_squared_error(y_test, y_pred_sofmax, squared=False)
softmax_r2 = r2_score(y_test, y_pred_sofmax)

print("MAE:", softmax_mae)
print("MSE:", softmax_mse)
print("RMSE:", softmax_rmse)
print("R2 Score:", softmax_r2)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


MAE: 103.20144462279293
MSE: 31105.540128410914
RMSE: 176.36762777905392
R2 Score: -0.4077697837064591


In [20]:
from sklearn.linear_model import LogisticRegression
softmax_reg = LogisticRegression(multi_class='multinomial', solver='saga')
softmax_reg.fit(X_train, y_train)
y_pred_sofmax = softmax_reg.predict(X_test)

softmax_mae = mean_absolute_error(y_test, y_pred_sofmax)
softmax_mse = mean_squared_error(y_test, y_pred_sofmax)
softmax_rmse = mean_squared_error(y_test, y_pred_sofmax, squared=False)
softmax_r2 = r2_score(y_test, y_pred_sofmax)

print("MAE:", softmax_mae)
print("MSE:", softmax_mse)
print("RMSE:", softmax_rmse)
print("R2 Score:", softmax_r2)

MAE: 103.52247191011236
MSE: 31094.408507223114
RMSE: 176.33606694951294
R2 Score: -0.40726599049511614


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


#Full Dataset

## Reading Data

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/AV_V/all_and_classification.csv')  # load data set


data = data.drop('run', axis=1)
data['forks'] = data['forks'].replace(['steady state'], 1)
data['forks'] = data['forks'].replace(['no steady state'], 0)
data = data.loc[data['forks'] == 1]
data = data.drop('forks', axis=1)


##Dividing data (train and test) based on file name 

In [ ]:
data_file_name = data['file_name']


In [ ]:
data_file_name_unique = data_file_name.drop_duplicates(keep="first")

In [ ]:
from sklearn.model_selection import train_test_split
data_file_name_train, data_file_name_test = train_test_split(data_file_name_unique, test_size=0.3,
                                                    random_state=1)

### Standardizing the data

In [ ]:
data_without_file_name_and_y = data.drop('file_name', axis=1)
data_without_file_name_and_y = data_without_file_name_and_y.drop('steady_state_starts', axis=1)
sc = StandardScaler()
sc.fit(data_without_file_name_and_y.T)
data_std = pd.DataFrame(sc.transform(data_without_file_name_and_y.T))
data_std = data_std.T

In [ ]:
data_std = data_std.merge(data_file_name, left_index=True, right_index=True)
data_std = data_std.merge(data['steady_state_starts'], left_index=True, right_index=True)

In [ ]:
train_data = pd.merge(data_std, data_file_name_train, left_on='file_name', right_on='file_name')

In [ ]:
test_data = pd.merge(data_std, data_file_name_test, left_on='file_name', right_on='file_name')

In [ ]:
train_data = train_data.drop('file_name', axis=1)
test_data = test_data.drop('file_name', axis=1)

In [ ]:
X_train = train_data.drop('steady_state_starts', axis=1) 
y_train  = train_data['steady_state_starts']
X_test = test_data.drop('steady_state_starts', axis=1)
y_test = test_data['steady_state_starts']

##Finding best hyperparameter values

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression
softmax_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs')
softmax_reg.fit(X_train, y_train)
y_pred_sofmax = softmax_reg.predict(X_test)

softmax_mae = mean_absolute_error(y_test, y_pred_sofmax)
softmax_mse = mean_squared_error(y_test, y_pred_sofmax)
softmax_rmse = mean_squared_error(y_test, y_pred_sofmax, squared=False)
softmax_r2 = r2_score(y_test, y_pred_sofmax)

print("MAE:", softmax_mae)
print("MSE:", softmax_mse)
print("RMSE:", softmax_rmse)
print("R2 Score:", softmax_r2)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


MAE: 465.15201192250373
MSE: 721729.5871833084
RMSE: 849.5466951164653
R2 Score: -0.6131716064369048


In [ ]:
from sklearn.linear_model import LogisticRegression
softmax_reg = LogisticRegression(multi_class='multinomial', solver='sag')
softmax_reg.fit(X_train, y_train)
y_pred_sofmax = softmax_reg.predict(X_test)

softmax_mae = mean_absolute_error(y_test, y_pred_sofmax)
softmax_mse = mean_squared_error(y_test, y_pred_sofmax)
softmax_rmse = mean_squared_error(y_test, y_pred_sofmax, squared=False)
softmax_r2 = r2_score(y_test, y_pred_sofmax)

print("MAE:", softmax_mae)
print("MSE:", softmax_mse)
print("RMSE:", softmax_rmse)
print("R2 Score:", softmax_r2)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


MAE: 496.9672131147541
MSE: 787582.7019374069
RMSE: 887.4585635044641
R2 Score: -0.7603629878119367


In [ ]:
from sklearn.linear_model import LogisticRegression
softmax_reg = LogisticRegression(multi_class='multinomial', solver='saga')
softmax_reg.fit(X_train, y_train)
y_pred_sofmax = softmax_reg.predict(X_test)

softmax_mae = mean_absolute_error(y_test, y_pred_sofmax)
softmax_mse = mean_squared_error(y_test, y_pred_sofmax)
softmax_rmse = mean_squared_error(y_test, y_pred_sofmax, squared=False)
softmax_r2 = r2_score(y_test, y_pred_sofmax)

print("MAE:", softmax_mae)
print("MSE:", softmax_mse)
print("RMSE:", softmax_rmse)
print("R2 Score:", softmax_r2)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


MAE: 473.0163934426229
MSE: 738641.6229508197
RMSE: 859.4426234198648
R2 Score: -0.6509724897478799
